In [1]:
!unzip flowers17.zip

Archive:  flowers17.zip
   creating: flowers17/
   creating: flowers17/bluebell/
  inflating: flowers17/bluebell/image_0241.jpg  
  inflating: flowers17/bluebell/image_0242.jpg  
  inflating: flowers17/bluebell/image_0243.jpg  
  inflating: flowers17/bluebell/image_0244.jpg  
  inflating: flowers17/bluebell/image_0245.jpg  
  inflating: flowers17/bluebell/image_0246.jpg  
  inflating: flowers17/bluebell/image_0247.jpg  
  inflating: flowers17/bluebell/image_0248.jpg  
  inflating: flowers17/bluebell/image_0249.jpg  
  inflating: flowers17/bluebell/image_0250.jpg  
  inflating: flowers17/bluebell/image_0251.jpg  
  inflating: flowers17/bluebell/image_0252.jpg  
  inflating: flowers17/bluebell/image_0253.jpg  
  inflating: flowers17/bluebell/image_0254.jpg  
  inflating: flowers17/bluebell/image_0255.jpg  
  inflating: flowers17/bluebell/image_0256.jpg  
  inflating: flowers17/bluebell/image_0257.jpg  
  inflating: flowers17/bluebell/image_0258.jpg  
  inflating: flowers17/bluebell/image

In [2]:
!pip install progressbar2

In [3]:
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from imutils import paths
import numpy as np
import progressbar
import argparse
import random
import os

In [4]:
import h5py
import os


class HDF5DatasetWriter:
    def __init__(self, dims, output_path, data_key="images", buf_size=1000):
        if os.path.exists(output_path):
            raise ValueError("The supplied 'output_path' already exists and cannot be overwritten. "
                             "Manually delete the file before continuing.", output_path)

        # Open HDF5 database for writing and create two datasets:
        # one to store the images/features and another to store the
        # class labels
        self.db = h5py.File(output_path, "w")
        self.data = self.db.create_dataset(data_key, dims, dtype="float")
        self.labels = self.db.create_dataset("labels", (dims[0],), dtype="int")

        # store buffer size then initialize the buffer itself
        # along with the index into the datasets
        self.buf_size = buf_size
        self.buffer = {"data": [], "labels": []}
        self.idx = 0

    def add(self, rows, labels):
        self.buffer["data"].extend(rows)
        self.buffer["labels"].extend(labels)

        if len(self.buffer["data"]) >= self.buf_size:
            self.flush()

    def flush(self):
        i = self.idx + len(self.buffer["data"])
        self.data[self.idx: i] = self.buffer["data"]
        self.labels[self.idx: i] = self.buffer["labels"]
        self.idx = i
        self.buffer = {"data": [], "labels": []}

    def store_class_labels(self, class_labels):
        dt = h5py.special_dtype(vlen=str)
        label_set = self.db.create_dataset("label_names", (len(class_labels),), dtype=dt)
        label_set[:] = class_labels

    def close(self):
        if len(self.buffer["data"]) > 0:
            self.flush()

        self.db.close()


In [5]:
args = {}
args["dataset"] = "/content/flowers17"
args["output"] = "/content/flowers17_features.hdf5"
args["batch_size"] = 32
args["buffer_size"] = 1000

bs = args["batch_size"]

In [6]:
print("[INFO] loading images...")
image_paths = list(paths.list_images(args["dataset"]))
random.shuffle(image_paths)

labels = [p.split(os.path.sep)[-2] for p in image_paths]
le = LabelEncoder()
labels = le.fit_transform(labels)

[INFO] loading images...


In [7]:
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=False)
dataset = HDF5DatasetWriter((len(image_paths), 512 * 7 * 7),
                            args["output"], data_key="features", buf_size=args["buffer_size"])
dataset.store_class_labels(le.classes_)

[INFO] loading network...
58889256/58889256 [==============================] - 0s 0us/step


In [8]:
widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(max_value=len(image_paths), widgets=widgets).start()

for i in np.arange(0, len(image_paths), bs):
    batch_paths = image_paths[i:i + bs]
    batch_labels = labels[i:i + bs]
    batch_images = []

    for (j, image_path) in enumerate(batch_paths):
        image = load_img(image_path, target_size=(224, 224))
        image = img_to_array(image)

        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)

        batch_images.append(image)

    batch_images = np.vstack(batch_images)
    features = model.predict(batch_images, batch_size=bs)

    # flatten features
    features = features.reshape((features.shape[0], 512 * 7 * 7))

    dataset.add(features, batch_labels)
    pbar.update(i)

dataset.close()
pbar.finish()

Extracting Features:   0% |                                    | ETA:  --:--:--

1/1 [==============================] - 11s 11s/step


Extracting Features:   0% |                                    | ETA:  --:--:--

1/1 [==============================] - 0s 37ms/step


Extracting Features:   2% |                                    | ETA:   0:07:57

...

Extracting Features:  94% |#################################   | ETA:   0:00:01

1/1 [==============================] - 0s 29ms/step


Extracting Features:  96% |##################################  | ETA:   0:00:00

1/1 [==============================] - 2s 2s/step


Extracting Features: 100% |####################################| Time:  0:00:27


In [12]:
db = h5py.File("/content/flowers17_features.hdf5")
print(list(db.keys()))
print(db['features'].shape)
print(db['labels'].shape)
print(db['label_names'].shape)

['features', 'label_names', 'labels']
(1360, 25088)
(1360,)
(17,)
